In [1]:
import nibabel as nib

import numpy as np

import scipy.stats as stats

from nilearn.image import resample_img

In [2]:
# Groups
groups=['HCP','CHIASM']

# Participants
CHIASM_con=['CON1','CON2','CON3','CON4','CON5','CON6','CON7','CON8']
CHIASM_alb=['ALB1','ALB2','ALB3','ALB4','ALB5','ALB6','ALB7','ALB8','ALB9']
HCP_con= ['101107','118730','131823','134223','151425','165436','208226','304727','379657','673455']
HCP=HCP_con

# Path to data
data_folder='/home/rjp/1_OVGU/2_Chiasmal_Abnormalities_Deep_Learning_Based_Segmentation/1_Data/'

In [3]:
# Function that takes dictionary and model and returns Sorensen-Dice coefficients for target-prediction pairs

# Sorensen-Dice coefficient
def dice(img_true, img_pred, smooth=0):       
    intersection = np.sum(img_true * img_pred)
    union = np.sum(img_true) + np.sum(img_pred)
    dice = (2. * intersection + smooth)/(union+smooth)   
    return dice
                    
def calculate_dice(path1, path2):
    file_target, file_prediction = nib.load(path1), nib.load(path2)   
    file_target_trans, file_prediction_trans = resample_img(file_target, file_prediction.affine, file_prediction.shape, 'linear'),resample_img(file_prediction, file_prediction.affine, file_prediction.shape, 'linear')       
    data_target, data_prediction = file_target_trans.get_fdata(), file_prediction_trans.get_fdata()
    return dice(data_target, data_prediction)

In [4]:
# DSC for X-mask_manual vs X-mask_CNN
dice_hand_refi_vs_pred={}

for group in ['CHIASM_con','CHIASM_alb', 'HCP_con']:
    dice_hand_refi_vs_pred[group]={}
    for sub in eval(group):
        hand=data_folder+'2_X-mask_manual/'+group[:-4]+'/'+sub+'/X-mask_manual.nii.gz'
        pred=data_folder+'5_X-mask_CNN/training_30ep_00025lr_dice/connectivity_3/threshold_1/'+group[:-4]+'/'+sub+'/X-mask_CNN_cropped_to_gt.nii.gz'    
        dice_hand_refi_vs_pred[group][sub]=calculate_dice(hand,pred)

In [5]:
# DSC for X-mask_manual vs X-mask_atlas-corrected
dice_hand_vs_refined={}

for group in ['CHIASM_con','CHIASM_alb', 'HCP_con']:
    dice_hand_vs_refined[group]={}
    for sub in eval(group):
        hand=data_folder+'2_X-mask_manual/'+group[:-4]+'/'+sub+'/X-mask_manual.nii.gz'
        label=data_folder+'4_X-mask_atlas-corrected/'+group[:-4]+'/'+sub+'/X-mask_atlas-corrected_cropped_to_gt.nii.gz'    
        dice_hand_vs_refined[group][sub]=calculate_dice(hand,label)

In [6]:
# DSC for X-mask_manual vs X-mask_atlas-initial
dice_hand_vs_initial={}

for group in ['CHIASM_con','CHIASM_alb', 'HCP_con']:
    dice_hand_vs_initial[group]={}
    for sub in eval(group):
        hand=data_folder+'2_X-mask_manual/'+group[:-4]+'/'+sub+'/X-mask_manual.nii.gz'
        label=data_folder+'3_X-mask_atlas-initial/'+group[:-4]+'/'+sub+'/X-mask_atlas-initial_cropped_to_gt.nii.gz'    
        dice_hand_vs_initial[group][sub]=calculate_dice(hand,label)

In [7]:
groups=['CHIASM_con','CHIASM_alb','HCP_con']

### Return mean and std for each type and group

In [8]:
# Values for initial
print('Initial')
for group in groups:
    print(f"{group} mean {np.mean(list(dice_hand_vs_initial[group].values())):.5f} and sem {stats.sem(list(dice_hand_vs_initial[group].values())):.5f}")

Initial
CHIASM_con mean 0.49794 and sem 0.04313
CHIASM_alb mean 0.52505 and sem 0.04560
HCP_con mean 0.57144 and sem 0.03460


In [9]:
# Values for refined
print('Refined')
for group in groups:
    print(f"{group} mean {np.mean(list(dice_hand_vs_refined[group].values())):.5f} and sem {stats.sem(list(dice_hand_vs_refined[group].values())):.5f}")

Refined
CHIASM_con mean 0.28217 and sem 0.03661
CHIASM_alb mean 0.33113 and sem 0.04004
HCP_con mean 0.74557 and sem 0.02951


In [10]:
# Values for prediction
print('Prediction')
for group in groups:
    print(f"{group} mean {np.mean(list(dice_hand_refi_vs_pred[group].values())):.5f} and sem {stats.sem(list(dice_hand_refi_vs_pred[group].values())):.5f}")

Prediction
CHIASM_con mean 0.74795 and sem 0.02572
CHIASM_alb mean 0.43704 and sem 0.08285
HCP_con mean 0.79464 and sem 0.02003


### Test the normality of distribution

In [11]:
# Normality of initial
print('Initial')
for g in groups:
    _, p = stats.normaltest(list(dice_hand_vs_initial[g].values()))
    print(f"{g} p-value equal {p}") # null hypothesis: sample comes from normal distribution

Initial
CHIASM_con p-value equal 0.8100766546397331
CHIASM_alb p-value equal 0.21287966781224515
HCP_con p-value equal 0.0055956934358798146


/home/rjp/Software/anaconda3/envs/optic_chiasm/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/home/rjp/Software/anaconda3/envs/optic_chiasm/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/home/rjp/Software/anaconda3/envs/optic_chiasm/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [12]:
# Normality of refined
print('Refined')
for g in groups:
    _, p = stats.normaltest(list(dice_hand_vs_refined[g].values()))
    print(f"{g} p-value equal {p}") # null hypothesis: sample comes from normal distribution

Refined
CHIASM_con p-value equal 0.6921551631793526
CHIASM_alb p-value equal 0.5146016505687968
HCP_con p-value equal 0.5118172081702516


In [13]:
# Normality of predicted
print('Predicted')
for g in groups:
    _, p = stats.normaltest(list(dice_hand_refi_vs_pred[g].values()))
    print(f"{g} p-value equal {p}") # null hypothesis: sample comes from normal distribution

Predicted
CHIASM_con p-value equal 0.7167169701938942
CHIASM_alb p-value equal 0.915239603006597
HCP_con p-value equal 0.08181400062000069


### Tests

In [14]:
# Dictionary of tests
tests_pvalues={}

In [15]:
# HCP_con initial vs HCP_con refined
tests_pvalues['HCPcon_initial_vs_refined']=stats.ranksums(list(dice_hand_vs_initial['HCP_con'].values()),list(dice_hand_vs_refined['HCP_con'].values()))[1]

In [16]:
# HCP_con initial vs CHIASM_con initial
tests_pvalues['HCPcon_initial_vs_CHIASMcon_initial']=stats.ranksums(list(dice_hand_vs_initial['HCP_con'].values()),list(dice_hand_vs_initial['CHIASM_con'].values()))[1]

In [17]:
# HCP_con refined vs HCP_con predicted
tests_pvalues['HCPcon_refined_vs_HCPcon_predicted']=stats.ttest_ind(list(dice_hand_vs_refined['HCP_con'].values()),list(dice_hand_refi_vs_pred['HCP_con'].values()))[1]

In [18]:
# HCP_con refined vs CHIASM_con refined
tests_pvalues['HCPcon_refined_vs_CHIASMcon_refined']=stats.ttest_ind(list(dice_hand_vs_refined['HCP_con'].values()),list(dice_hand_vs_refined['CHIASM_con'].values()))[1]

In [19]:
# HCP_con predicted vs CHIASM_con predicted
tests_pvalues['HCPcon_predicted_vs_CHIASMcon_predicted']=stats.ttest_ind(list(dice_hand_refi_vs_pred['HCP_con'].values()),list(dice_hand_refi_vs_pred['CHIASM_con'].values()))[1]

In [20]:
# HCP_con predicted vs CHIASM_alb predicted
tests_pvalues['HCPcon_predicted_vs_CHIASMalb_predicted']=stats.ttest_ind(list(dice_hand_refi_vs_pred['HCP_con'].values()),list(dice_hand_refi_vs_pred['CHIASM_alb'].values()))[1]

In [21]:
# CHIASM_con initial vs CHIASM_con refined
tests_pvalues['CHIASMcon_initial_vs_CHIASMcon_refined']=stats.ttest_ind(list(dice_hand_vs_initial['CHIASM_con'].values()),list(dice_hand_vs_refined['CHIASM_con'].values()))[1]

In [22]:
# CHIASM_con refined vs CHIASM_con predicted
tests_pvalues['CHIASMcon_refined_vs_CHIASMcon_predicted']=stats.ttest_ind(list(dice_hand_vs_refined['CHIASM_con'].values()),list(dice_hand_refi_vs_pred['CHIASM_con'].values()))[1]

In [23]:
# CHIASM_con predicted vs CHIASM_alb predicted
tests_pvalues['CHIASMcon_predicted_vs_CHIASMalb_predicted']=stats.ttest_ind(list(dice_hand_refi_vs_pred['CHIASM_con'].values()),list(dice_hand_refi_vs_pred['CHIASM_alb'].values()))[1]

In [24]:
# CHIASM_alb refined vs CHIASM_alb predicted
tests_pvalues['CHIASMalb_refined_vs_CHIASMalb_predicted']=stats.ttest_ind(list(dice_hand_vs_refined['CHIASM_alb'].values()),list(dice_hand_refi_vs_pred['CHIASM_alb'].values()))[1]

In [25]:
tests_pvalues

{'HCPcon_initial_vs_refined': 0.002496908915141548,
 'HCPcon_initial_vs_CHIASMcon_initial': 0.10974463874701328,
 'HCPcon_refined_vs_HCPcon_predicted': 0.18577671131830137,
 'HCPcon_refined_vs_CHIASMcon_refined': 2.841531655413912e-08,
 'HCPcon_predicted_vs_CHIASMcon_predicted': 0.16486050363710597,
 'HCPcon_predicted_vs_CHIASMalb_predicted': 0.0003859639434391207,
 'CHIASMcon_initial_vs_CHIASMcon_refined': 0.0018978797309722537,
 'CHIASMcon_refined_vs_CHIASMcon_predicted': 5.6710212150268804e-08,
 'CHIASMcon_predicted_vs_CHIASMalb_predicted': 0.003959957457813841,
 'CHIASMalb_refined_vs_CHIASMalb_predicted': 0.26665686634201213}

In [ ]:
pvalues=(statsmodels.stats.multitest.multipletests(list(tests_pvalues.values()), alpha=0.05, method='bonferroni')[1]
my_formatted_list = [ '%.8f' % elem for elem in pvalues ]
my_formatted_list